# py-限定参数类型

In [5]:
import inspect
from functools import wraps

def typeassert(*ty_args, **ty_kws):
    def decorate(func):
        if not __debug__:
            return func
        sig = inspect.signature(func)
        bound_types = sig.bind_partial(*ty_args, **ty_kws).arguments
        
        @wraps(func)
        def wrapper(*args, **kws):
            bound_values = sig.bind(*args, **kws)
            for name, value in bound_values.arguments.items():
                if name in bound_types:
                    if not isinstance(value, bound_types[name]):
                        raise TypeError('Argument {} must be {}'.format(name, bound_types[name]))
            return func(*args, **kws)
        return wrapper
    return decorate


@typeassert(level=int, msg=bytes)
def handle_on_msg(level, msg):
    if level <= 20:
        print(msg.decode())
        
handle_on_msg('10', '123')

TypeError: Argument level must be <class 'int'>